# Intro Scalable Deep Learning: Day 3, Tutorial 1

**Content creators**: Mehdi Cherti, Jenia Jitsev

**Content reviewers / testers**: Helmholtz AI team

**Content supervisors** : Jenia Jitsev

# Tutorial Objectives

In distributed data parallel training, there are two key aspects that are key to consider when scaling to multiple workers. The first is aspect is **throughput** and the second is **training/generalization performance**.
We need to maintain a good **throughput** to make sure we use the hardware capability fully, even in the presence of internode communication (e.g., to aggregate gradients computed locally on different GPUs and use those to synchronize weight update step among the workers).
Maintaining the throughput high will result in faster training as we scale the number of workers.
However, maintaining throughput as the number of workers is scaled is not enough, because in the end, in machine learning, we ultimately care about generalization performance of the model. We will see in **Day 4** that using very large batch sizes (in data parallel training, scaling the number of GPUs result in scaling the effective batch size) can result in a number of issues in terms of training and/or generalization performance.
In this tutorial, we will concentrate only the first aspect, the throughput.

The throughput is measured as the number of examples processed per second, where "processing" includes the full pipeline: 

- Data loading
- Local forward pass
- Local backward pass
- Synchronization of the aggregated gradients across all GPUs
- Update of the neural network parameters

We would like to maintain a high throughput. If $K$ is the number of workers (GPUs), and $\tau_1$ is the throughput for $K=1$, in the ideal case we want $\tau_K^{*} = \tau_1 \times K$. In practice, the ideal throughput $\tau_K^{*}$ is never reached exactly, but the goal is to maintain it as high as possible. There are three related metrics that are interesting to look at:

- The raw throughput $\tau_K$ for $K$ workers (GPUs).
- The speedup $\frac{\tau_K}{\tau_1}$ for $K$ workers (GPUs) related to the reference throughput $\tau_1$.
We note that the ideal speedup is $\frac{\tau_K^{*}}{\tau_1} = K$.
- The efficiency $100\% \times \frac{\tau_K}{\tau_K^*}$. We note that the ideal efficiency is $100\%$.

In this tutorial, you wil learn how to:

- Run benchmarks to measure the throughput
- Analyze the effect of the number of nodes, the number of GPUs per node, the local batch size on the throughput

# Setup

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def plot_scaling_and_efficiency(df):
    """
    Render scaling and efficiency plots
    
    - Scaling plot is the relationship between the number of GPUs used
      and the speedup in the throughput
    - Efficiency is the ratio of throughput to ideal reference throughput
    """
    df = df.copy()
    # Number of GPUs per run
    df["GPUs"] = df["NODES"] * df["GPUS_PER_NODE"]
    df = df.sort_values(by="GPUs")
    
    # reference number of GPUs is the run with minimal number of GPUs
    refrence_nb_gpus = df.iloc[0]['GPUs']
    # reference throughput is the throughput of the configuration with the reference number of GPUs
    reference_throughput = df.iloc[0]["TOTAL_IMAGES_PER_SEC"]
    df["Speedup"] = df["TOTAL_IMAGES_PER_SEC"] / reference_throughput
    # efficiency is the  $E(K) = 100 \times \frac{T(K)}{K \times T(1)}$, where:
    # - $K$ is the number of GPUs
    # - $T(K)$ is the throughput for $K$ GPUs
    # - $T(1)$ the reference throughput (for $K=1$)
    df['efficiency'] =  df["TOTAL_IMAGES_PER_SEC"] / ( (df["GPUs"]/refrence_nb_gpus) * reference_throughput)
    
    # plot
    fig, ax1 = plt.subplots() 
    ax1.plot(df.GPUs, df.Speedup, 'go--', label='speedup', color='blue')
    ax1.set_xlabel('GPUs')
    ax1.set_ylabel('Speedup',color='blue')
    ax1.set_xticks(df.GPUs)
    ax1.plot(df.GPUs, df.GPUs / refrence_nb_gpus, 'go-', label='ideal speedup', color='orange')
    ax1.tick_params(axis='y', labelcolor='blue')
    scale = df.Speedup.max() * 1.5
    top = 0
    text_spacing = 0.1
    y = df['efficiency'] * 100#* scale + top
    ax2 = ax1.twinx() 
    ax2.set_ylabel('Efficiency (%)', color='red')
    ax2.plot(df.GPUs, y, c='red', marker="o", label='efficiency')
    ax2.tick_params(axis='y', labelcolor='red')
    for nb_gpus, effval, yval in zip(df.GPUs, df['efficiency'], y):
        ax2.text(nb_gpus, yval+text_spacing, f"{effval*100:.2f}%", size=11, c='red')

def plot_images_per_sec(df):
    """
    Render raw images per sec plot
    """
    df = df.copy()
    df["GPUs"] = df["NODES"] * df["GPUS_PER_NODE"]
    df = df.sort_values(by="GPUs")
    df.plot(x="GPUs",  y="TOTAL_IMAGES_PER_SEC", color='pink', edgecolor='red', kind='bar')
    
def plot_images_per_sec_per_batch_size(df):
    df = df.copy()
    df["GPUs"] = df["NODES"] * df["GPUS_PER_NODE"]
    df = df.sort_values(by="GPUs")
    df.plot(x="LOCAL_BATCH_SIZE",  y="TOTAL_IMAGES_PER_SEC", color='pink', edgecolor='red', kind='bar')
    ticks = [f"({int(row['LOCAL_BATCH_SIZE'])})x({int(row['NODES'])})x({int(row['GPUS_PER_NODE'])})" for _, row in df.iterrows()]
    plt.xticks(range(len(df)), ticks)
    plt.xlabel("B_ref x Nodes x GPUs per node")

### Exercise 1
* Goal: run a tinyImageNet training on multiple nodes and plot scaling performance with respect to Image/s throughput

#### Task 1.1 : Distributed training on tinyImageNet, computing the image throughput
* Take the code `train.py`, adapt it for Horovod distributed training, and display the throughput (images per second). In order to do so, please fill all the **TODO** parts in `train.py`.
* Hint: to execute the code, you can run it with `source /p/project/training2306/software_environment/activate.sh
;export CUDA_VISIBLE_DEVICES="0,1,2,3"; srun -N 1 -n 4 --ntasks-per-node=4 --account=training2306 --time 00:05:00 --gres=gpu:4 --partition=dc-gpu python -u train.py --batch_size=16`. Alternatively, you can also adapt `distributed-job-skeleton.sh` to submit a job via job script.

In the following we will use `benchmark.sh` to conduct systematic experiments with differnet configurations.
Before using it, please give execute permissions:

In [1]:
!chmod +x benchmark.sh

#### Task 1.2 : Observing the image throughput scaling while varying worker's number

The goal is  to execute different training runs with various number of workers, e.g. $K=1,2,4,8$ while protocoling Images/s for each run. We will choose a fixed batch size per worker.

- Please first fill `config_task1.2.csv`  accordingly to test the different configurations. See also `config_example.csv` to see a config example. `config_task1.2.csv` should contain 5 lines (the header + 4 lines describing the different configurations):
    - nodes=1 and gpus_per_node=1 ($K=1$)
    - nodes=1 and gpus_per_node=2 ($K=2$)
    - nodes=1 and gpus_per_node=4 ($K=4$)
    - nodes=2 and gpus_per_node=4 ($K=8$)
    - The local batch size should be always fixed in those 4 different configurations, you can try 256 or 512 or 1024.
    - Feel free to try nodes > 2 (4, 8) - be aware that this will go into the standard job queue and may result in longer waiting time before job is executed. You can do those experiments with larger node numbers as a homework.
- After modifying `config_1.2.csv`, run the benchmark using `./benchmark.sh config_task1.2.csv results_task1.2.csv` to generate a CSV
     - How long is it necessary to run a training for estimating Images/s throughput?
- Use the plotting procedure `plot_scaling_and_efficiency` and `plot_images_per_sec` to display the results (see the cell below).
    - What can you say about the observed scaling behavior? Are there any issues when choosing a very small or very large local reference batch size $B_{\text{ref}}$ per worker?

In [ ]:
df = pd.read_csv('results_task1.2.csv')
df

In [ ]:
plot_images_per_sec(df)
plot_scaling_and_efficiency(df)

### Exercise 2
* Goal: Use scaling plot routines established in the exercise earlier to observer how scaling performance varies depending on the effective global batch size $\mathfrak{B}$, when using different number of workers and systematically varying local reference batch size $B_{ref}$

#### Task 2.1 : Scaling for same effective batch size $\mathfrak{B}$ using distinct $\vert B_{\text{ref}} \vert$ and worker number $K$ 
- Choose different combinations of $\vert B_{\text{ref}} \vert$ and $K$ that result in the same $\mathfrak{B} = K B_{\text{ref}}$. As an example, fix $\mathfrak{B}$ to 4096 in all the runs, vary $K$ and $B_{\text{ref}}$ accordingly. Use $K \in \{2,4,8\}$ (which will be determined by the number of nodes and number of gpus per node, see **Task 1.2**), and for each $K$, use $B_{\text{ref}} = \frac{\mathfrak{B}}{K}$. Example of configurations for $\mathfrak{B} = 4096$:
    - nodes=1 and gpus_per_node=2 ($K=2$), with $B_{\text{ref}} = 2048$
    - nodes=2 and gpus_per_node=1 ($K=2$), with $B_{\text{ref}} = 2048$
    - nodes=1 and gpus_per_node=4 ($K=4$), with $B_{\text{ref}} = 1024$
    - nodes=2 and gpus_per_node=4 ($K=8$), with $B_{\text{ref}} = 512$
- Using the chosen configurations, fill `config_task2.1.csv`, and then run the benchmark using `./benchmark.sh config_task2.1.csv results_task2.1.csv` to generate a CSV
    - What differences do you expect to encounter?
- Use the plotting procedure `plot_images_per_sec_per_batch_size` to display the results (see the cell below). 
    - Observe scaling behavior and see how it matches your expectations

In [ ]:
df = pd.read_csv('results_task2.1.csv')
df

In [ ]:
plot_images_per_sec_per_batch_size(df)

#### Task 2.2 : Observing Scaling across different $\vert B_{\text{ref}} \vert$ and worker numbers $K$
The goal is to conduct a systematic series of runs through a range of $\vert B_{\text{ref}} \vert = small - large$ and $K = 1, 8$. 

- Fill `config_task2.2.csv` by trying $K=1$ and $K=8$ GPUs for different reference batch sizes  $B_{\text{ref}} \in \{128,512,2048\}$. In total, `config_task2.2.csv` should contain 7 lines (header + 6 configurations).
- Run the benchmark using `./benchmark.sh config_task2.2.csv results_task2.2.csv` to generate a CSV
- Use the plotting procedure `plot_scaling_and_efficiency` to display the results (see the cell below).
    - Are there any issues with scaling for certain configurations?
    - What are configurations that give scaling that looks reasonable?
    - What are the factors behind the differences?
- Feel free to try nodes > 2 (4, 8) - be aware that this will go into the standard job queue and may result in longer waiting time before job is executed. You can do those experiments with larger node numbers as a homework.

In [ ]:
df = pd.read_csv('results_task2.2.csv')
df

In [ ]:
plot_scaling_and_efficiency(df[df.LOCAL_BATCH_SIZE==128])
plot_scaling_and_efficiency(df[df.LOCAL_BATCH_SIZE==512])
plot_scaling_and_efficiency(df[df.LOCAL_BATCH_SIZE==2048])

# Summary

In this tutorial we concentrated on benchmarking and analysing the throughput, which is key in data parallel distributed training.
While it is difficult to reach an ideal throughput when scaling the number of nodes, the goal is to approach it as much as possible by
avoiding data loading bottlenecks, giving enough work to the GPUs to utilize them fully, and minimizing communication costs
as much as possible.